# 誤差逆伝播法に対応したNNを実装する

In [3]:
import sys, os
import numpy as np
from collections import OrderedDict
from common.layers import SoftmaxWithLoss, Affine, ReLU

### [演習]
* 以下の誤差逆伝播法に対応したNNクラスを完成させましょう。

In [4]:
# ヒント

# OrderedDictを用いると、順序を保持したdictを生成することができる
# ただし、Pythonバージョン3.6以降は、普通のdictでも順序が保存される
dic1 = OrderedDict()
dic1["a"] = 1
dic1["c"] = 3
dic1["b"] = 2
for key, value in dic1.items():
    print(key, value)
print()

# dictから逆順でvalueを取り出す方法
layers = list(dic1.values())
layers.reverse() # 順序を逆転させる
print(layers)

a 1
c 3
b 2

[2, 3, 1]


In [5]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict() # 順番付きdict形式. 
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss() # 出力層
        
    def predict(self, x):
        """
        予測関数
        x : 入力
        """
        for layer in self.layers.values():
            # 入力されたxを更新していく = 順伝播計算
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        """
        損失関数
        x:入力データ, t:正解データ
        """
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        """
        正解率
        """
        # 予測 返り値は正規化されていない実数
        y = self.predict(x)
        
        #正規化されていない実数をもとに、最大値になるindexに変換する
        y = np.argmax(y, axis=1)
        
        if t.ndim != 1 : 
            """
            one-hotベクトルの場合、正解データをindexに変換する
            """
            t = np.argmax(t, axis=1)
        
        # 正解率
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def gradient(self, x, t):
        """
        全パラメータの勾配を計算
        """
        
        # 順伝播
        self.loss(x, t)

        # 逆伝播
        dout = self.lastLayer.backward() # 出力層
        
        ## doutを逆向きに伝える 
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # dW, dbをgradsにまとめる
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [6]:
# 学習用データ
x = np.array([[1, 2, 3],
            [3, 2, 1]])
t = np.array([[0, 0, 1],
            [1, 0, 0]])

tnet = TwoLayerNet( input_size=3, hidden_size=5, output_size=3)

print("predict=\n", tnet.predict(x))
print()
print("loss=\n", tnet.loss(x, t))
print()
print("accuracy=", tnet.accuracy(x, t))
print()
print("gradient=\n",)
grads = tnet.gradient(x, t)
for key, value in grads.items():
    print(key)
    print(value)

predict=
 [[-3.97269175e-04  1.59953224e-04 -2.19746877e-05]
 [-8.03456651e-04  7.20340402e-04 -2.19729341e-05]]

loss=
 1.098964084486718

accuracy= 0.0

gradient=

W1
[[ 0.00778857 -0.00396958  0.01040611 -0.00562248  0.01609339]
 [ 0.01054678 -0.00793915  0.00306116 -0.00374832  0.01131719]
 [ 0.01330499 -0.01190873 -0.0042838  -0.00187416  0.00654098]]
b1
[ 0.00527339 -0.00396958  0.00153058 -0.00187416  0.00565859]
W2
[[ 0.0007528   0.00286956 -0.00362236]
 [ 0.00177439  0.00177538 -0.00354977]
 [-0.00347265  0.00588921 -0.00241656]
 [-0.00104308  0.00052173  0.00052134]
 [-0.01694857  0.01200097  0.00494761]]
b2
[-0.16684652  0.33350031 -0.16665378]
